In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission

import sys

import tensorflow as tf
from tensorflow import keras

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

sys.path.append('../')

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/pyt

In [2]:
# word_vector_size = w2v_model.wv.vectors.shape[1]
word_vector_size = 300
word_vector_size

300

In [3]:
# This is unfortunately very high memory consumptive

# Test here is not the unlabelled testset, but rather the 0.8 split of the original training set

sentence_train_x = np.load("np_arrays/sentences_train_x_08_google.npy")
sentence_train_y = np.load("np_arrays/sentences_train_y_08_google.npy")

sentence_test_x = np.load("np_arrays/sentences_train_x_02_google.npy")
sentence_test_y = np.load("np_arrays/sentences_train_y_02_google.npy")

In [4]:
# Just print how many 0 we have, just to be sure that we have shuffled the dataset
y_ = np.argmax(sentence_train_y, axis=-1)
len(y_[y_==0])

999771

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
# now perform training on the new features vectors.

# Build a "deep" neural network with 2 hidden layers. When we see that it somehow works,
# we can start doing some cross validation on it.

for layer_size in range(30, 101, 5):
    
    print("\n\nStarting with number of layers: {}".format(layer_size))
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                  )

    filepath="models/weights_{}_layers_of_size_{}_big".format(len(model.layers), layer_size)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit(x=sentence_train_x,
              y=sentence_train_y,
              validation_data=(sentence_test_x,  sentence_test_y),
              callbacks=callbacks_list,
              epochs=40,
              use_multiprocessing=True,
              batch_size=128)
        




Starting with number of layers: 30
Instructions for updating:
Colocations handled automatically by placer.


2019-12-18 05:02:35,143 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Train on 2000000 samples, validate on 500000 samples
Instructions for updating:
Use tf.cast instead.


2019-12-18 05:02:35,260 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/40
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.4918 - acc: 0.7532
Epoch 00001: val_acc improved from -inf to 0.76045, saving model to models/weights_3_layers_of_size_30_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4918 - acc: 0.7532 - val_loss: 0.4801 - val_acc: 0.7605
Epoch 2/40
1997824/2000000 [============================>.] - ETA: 0s - loss: 0.4783 - acc: 0.7622
Epoch 00002: val_acc improved from 0.76045 to 0.76227, saving model to models/weights_3_layers_of_size_30_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4783 - acc: 0.7622 - val_loss: 0.4788 - val_acc: 0.7623
Epoch 3/40
1997184/2000000 [============================>.] - ETA: 0s - loss: 0.4741 - acc: 0.7647
Epoch 00003: val_acc improved from 0.76227 to 0.76427, saving model to models/weights_3_layers_of_size_30_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4741 - acc: 0.7647 - val_loss: 0.

...

Epoch 2/40
1997056/2000000 [============================>.] - ETA: 0s - loss: 0.4718 - acc: 0.7664
Epoch 00002: val_acc improved from 0.76389 to 0.76664, saving model to models/weights_3_layers_of_size_90_big
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.4718 - acc: 0.7664 - val_loss: 0.4709 - val_acc: 0.7666
Epoch 3/40
1998976/2000000 [============================>.] - ETA: 0s - loss: 0.4664 - acc: 0.7694
Epoch 00003: val_acc improved from 0.76664 to 0.76911, saving model to models/weights_3_layers_of_size_90_big
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.4664 - acc: 0.7694 - val_loss: 0.4675 - val_acc: 0.7691
Epoch 4/40
1998976/2000000 [============================>.] - ETA: 0s - loss: 0.4631 - acc: 0.7715
Epoch 00004: val_acc improved from 0.76911 to 0.77085, saving model to models/weights_3_layers_of_size_90_big
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.4631 - acc: 0.7715 - val_loss:

Epoch 29/40
1996800/2000000 [============================>.] - ETA: 0s - loss: 0.4417 - acc: 0.7843
Epoch 00029: val_acc did not improve from 0.77508
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.4417 - acc: 0.7843 - val_loss: 0.4601 - val_acc: 0.7746
Epoch 30/40
1999104/2000000 [============================>.] - ETA: 0s - loss: 0.4414 - acc: 0.7845
Epoch 00030: val_acc improved from 0.77508 to 0.77542, saving model to models/weights_3_layers_of_size_90_big
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.4414 - acc: 0.7845 - val_loss: 0.4589 - val_acc: 0.7754
Epoch 31/40
1997824/2000000 [============================>.] - ETA: 0s - loss: 0.4410 - acc: 0.7850
Epoch 00031: val_acc did not improve from 0.77542
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.4410 - acc: 0.7850 - val_loss: 0.4610 - val_acc: 0.7749
Epoch 32/40
1996544/2000000 [============================>.] - ETA: 0s - loss: 0.4407 - acc

Epoch 15/40
1996928/2000000 [============================>.] - ETA: 0s - loss: 0.4479 - acc: 0.7808
Epoch 00015: val_acc improved from 0.77431 to 0.77477, saving model to models/weights_3_layers_of_size_95_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4479 - acc: 0.7808 - val_loss: 0.4587 - val_acc: 0.7748
Epoch 16/40
1996928/2000000 [============================>.] - ETA: 0s - loss: 0.4471 - acc: 0.7810
Epoch 00016: val_acc did not improve from 0.77477
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4472 - acc: 0.7810 - val_loss: 0.4583 - val_acc: 0.7734
Epoch 17/40
1999232/2000000 [============================>.] - ETA: 0s - loss: 0.4465 - acc: 0.7815
Epoch 00017: val_acc did not improve from 0.77477
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4465 - acc: 0.7815 - val_loss: 0.4588 - val_acc: 0.7740
Epoch 18/40
1999104/2000000 [============================>.] - ETA: 0s - loss: 0.4460 - acc

1999872/2000000 [============================>.] - ETA: 0s - loss: 0.4713 - acc: 0.7665
Epoch 00002: val_acc improved from 0.76531 to 0.76532, saving model to models/weights_3_layers_of_size_100_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4713 - acc: 0.7665 - val_loss: 0.4701 - val_acc: 0.7653
Epoch 3/40
1999744/2000000 [============================>.] - ETA: 0s - loss: 0.4660 - acc: 0.7697
Epoch 00003: val_acc improved from 0.76532 to 0.76878, saving model to models/weights_3_layers_of_size_100_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4660 - acc: 0.7697 - val_loss: 0.4668 - val_acc: 0.7688
Epoch 4/40
1998336/2000000 [============================>.] - ETA: 0s - loss: 0.4625 - acc: 0.7719
Epoch 00004: val_acc improved from 0.76878 to 0.77046, saving model to models/weights_3_layers_of_size_100_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4625 - acc: 0.7719 - val_loss: 0.4644 

Epoch 29/40
1999104/2000000 [============================>.] - ETA: 0s - loss: 0.4403 - acc: 0.7855
Epoch 00029: val_acc did not improve from 0.77552
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4403 - acc: 0.7855 - val_loss: 0.4581 - val_acc: 0.7755
Epoch 30/40
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.4400 - acc: 0.7858
Epoch 00030: val_acc improved from 0.77552 to 0.77560, saving model to models/weights_3_layers_of_size_100_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4400 - acc: 0.7858 - val_loss: 0.4579 - val_acc: 0.7756
Epoch 31/40
1997696/2000000 [============================>.] - ETA: 0s - loss: 0.4396 - acc: 0.7858
Epoch 00031: val_acc did not improve from 0.77560
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.4396 - acc: 0.7858 - val_loss: 0.4602 - val_acc: 0.7751
Epoch 32/40
1996160/2000000 [============================>.] - ETA: 0s - loss: 0.4393 - ac

## To execute this after having trained the above model

After having decided which one is the best model to use, just load the correct model, set the parameters of the model_star so that it matches the one of the best model, and compute the results!

(The below part needs to be rewritten to take advantage of the python modules)

In [ ]:
# Recreate exactl the best model
model_star = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])

model_star.load_weights('models/weights_2_layers_100_big')

model_star.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
print(model_star)

In [ ]:
# Test it on the test dataset to see if it gives the same best results.
model_star.evaluate(sentence_test_x,  sentence_test_y, verbose=2)[1]
model_star.predict(sentence_test_x)

## GET the test data and predict them

In [5]:
# For non binary models
w2v_model = Word2Vec.load("models/w2v_model_epochs_5_win_5_cbow_300.model")

# For binary models
#w2v_model = gensim.models.KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin', binary=True)

2019-12-16 13:43:05,152 : INFO : loading Word2Vec object from models/w2v_model_epochs_5_win_5_cbow_300.model
2019-12-16 13:43:05,885 : INFO : loading wv recursively from models/w2v_model_epochs_5_win_5_cbow_300.model.wv.* with mmap=None
2019-12-16 13:43:05,885 : INFO : loading vectors from models/w2v_model_epochs_5_win_5_cbow_300.model.wv.vectors.npy with mmap=None
2019-12-16 13:43:06,932 : INFO : setting ignored attribute vectors_norm to None
2019-12-16 13:43:06,933 : INFO : loading vocabulary recursively from models/w2v_model_epochs_5_win_5_cbow_300.model.vocabulary.* with mmap=None
2019-12-16 13:43:06,933 : INFO : loading trainables recursively from models/w2v_model_epochs_5_win_5_cbow_300.model.trainables.* with mmap=None
2019-12-16 13:43:06,934 : INFO : loading syn1neg from models/w2v_model_epochs_5_win_5_cbow_300.model.trainables.syn1neg.npy with mmap=None
2019-12-16 13:43:07,366 : INFO : setting ignored attribute cum_table to None
2019-12-16 13:43:07,367 : INFO : loaded models/w

In [ ]:
test_df_cleaned = pd.read_pickle("dataframes/test_cleaned.pickle")

In [ ]:
test_df_cleaned.head()

In [ ]:
sentence_submission_x = create_sentence_vectors_submission(test_df_cleaned['sentence'], word_vector_size, w2v_model)

In [ ]:
sentence_submission_x.shape

In [ ]:
model_star.predict(sentence_submission_x)

In [ ]:
predictions = []
for el in model_star.predict(sentence_submission_x):
    predictions.append(-1 if el[0] > el[1] else 1)

print(predictions[:10])

results = pd.DataFrame({
    "Id": test_df_cleaned['label'],
    "Prediction": predictions
})

results.head(20)

In [ ]:
results.to_csv('Submission.csv', index=False)